In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import requests
import os

ADS_API_URL = "https://api.adsabs.harvard.edu/v1/search/query"

def get_citation_count(paper_title, max_results=5):
    """
    Query ADS for a paper by title and return citation count(s).
    """
    token = os.getenv("ADS_API_TOKEN")
    if token is None:
        raise RuntimeError("ADS_API_TOKEN environment variable not set")

    headers = {
        "Authorization": f"Bearer {token}"
    }

    params = {
        "q": f'title:"{paper_title}"',
        "fl": "title,author,year,citation_count,bibcode",
        "rows": max_results
    }

    response = requests.get(ADS_API_URL, headers=headers, params=params)
    response.raise_for_status()

    data = response.json()
    papers = data["response"]["docs"]

    if not papers:
        print("No papers found.")
        return []

    results = []
    for p in papers:
        results.append({
            "title": p.get("title", [""])[0],
            "year": p.get("year"),
            "authors": ", ".join(p.get("author", [])[:3]) + (" et al." if len(p.get("author", [])) > 3 else ""),
            "citation_count": p.get("citation_count", 0),
            "bibcode": p.get("bibcode")
        })

    return results
